# Apache Sedona Example Notebook

This notebook demonstrates basic Apache Sedona functionality for spatial data processing.

## Setup and Initialization

In [10]:
from pyspark.sql import SparkSession
from sedona.register import SedonaRegistrator
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

In [11]:
# Initialize Spark with Sedona
spark = SparkSession.builder \
    .appName("SedonaExample") \
    .config("spark.serializer", KryoSerializer.getName) \
    .config("spark.kryo.registrator", SedonaKryoRegistrator.getName) \
    .config('spark.jars.packages',
            'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.4.1,'
            'org.apache.sedona:sedona-viz-3.0_2.12:1.4.1,'
            'org.apache.sedona:sedona-python-adapter-3.0_2.12:1.4.1') \
    .getOrCreate()

# Register Sedona functions
SedonaRegistrator.registerAll(spark)

/tmp/ipykernel_24/544328490.py:13: DeprecationWarning: Call to deprecated function registerAll (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  SedonaRegistrator.registerAll(spark)
/usr/local/lib/python3.9/dist-packages/sedona/register/geo_registrator.py:45: DeprecationWarning: Call to deprecated function register (Deprecated since 1.4.1, use SedonaContext.create() instead.).
  cls.register(spark)
25/10/24 13:24:41 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/10/24 13:24:41 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
25/10/24 13:24:41 WARN UdtRegistratorWrapper$: Geotools was not found on the classpath. Raster type will not be registered.
25/10/24 13:24:41 WARN SimpleFunctionRegistry: The function st_union_aggr replaced a previously registered function.
25/10/24 13:24:41 WARN SimpleFunctionRegistry: The function st_envelope_aggr re

True

## Troubleshooting

If you encounter `NoClassDefFoundError: org/opengis/referencing/NoSuchAuthorityCodeException`, this means the GeoTools dependencies are missing. 

**Solution:**
1. Rebuild the Docker image with the updated Dockerfile that includes GeoTools JARs
2. Or manually add the required JARs to the Spark classpath

**Manual fix (if needed):**
```bash
# From within the container
wget -P /opt/spark/jars/ https://repo1.maven.org/maven2/org/geotools/gt-referencing/29.2/gt-referencing-29.2.jar
wget -P /opt/spark/jars/ https://repo1.maven.org/maven2/org/geotools/gt-opengis/29.2/gt-opengis-29.2.jar
```

## Creating Spatial Data

In [12]:
# Create sample spatial data
sample_points = spark.sql("""
    SELECT 
        ST_Point(CAST(RAND() * 360 - 180 AS DECIMAL(10,6)), 
                 CAST(RAND() * 180 - 90 AS DECIMAL(10,6))) as geometry,
        CAST(RAND() * 100 AS INT) as value,
        'point_' || CAST(RAND() * 1000 AS INT) as name
    FROM range(100)
""")

sample_points.show(5)

+--------------------+-----+---------+
|            geometry|value|     name|
+--------------------+-----+---------+
|POINT (-152.85212...|   69|point_860|
|POINT (135.211903...|   83| point_69|
|POINT (-153.48906...|   43|point_409|
|POINT (-124.40813...|   77|point_327|
|POINT (-153.35868...|   16|point_426|
+--------------------+-----+---------+
only showing top 5 rows



## Spatial Operations

In [13]:
# Create a polygon for spatial filtering
bounding_box = spark.sql("""
    SELECT ST_PolygonFromEnvelope(-10.0, -10.0, 10.0, 10.0) as bbox
""")

# Filter points within the bounding box
sample_points.createOrReplaceTempView("points")
bounding_box.createOrReplaceTempView("bbox")

filtered_points = spark.sql("""
    SELECT p.*, ST_X(p.geometry) as longitude, ST_Y(p.geometry) as latitude
    FROM points p, bbox b
    WHERE ST_Within(p.geometry, b.bbox)
""")

print(f"Total points: {sample_points.count()}")
print(f"Points within bounding box: {filtered_points.count()}")
filtered_points.show()

Total points: 100
Points within bounding box: 0
+--------+-----+----+---------+--------+
|geometry|value|name|longitude|latitude|
+--------+-----+----+---------+--------+
+--------+-----+----+---------+--------+



## Spatial Analysis

In [14]:
# Calculate distances between points
distance_analysis = spark.sql("""
    SELECT 
        p1.name as point1,
        p2.name as point2,
        ST_Distance(p1.geometry, p2.geometry) as distance
    FROM points p1
    CROSS JOIN points p2
    WHERE p1.name != p2.name
    ORDER BY distance
    LIMIT 10
""")

print("Closest point pairs:")
distance_analysis.show()

Closest point pairs:


25/10/24 13:25:03 ERROR Executor: Exception in task 6.0 in stage 48.0 (TID 195)
java.lang.NoClassDefFoundError: org/opengis/referencing/NoSuchAuthorityCodeException
	at org.apache.spark.sql.sedona_sql.expressions.ST_Distance$$anonfun$$lessinit$greater$1.apply(Functions.scala:39)
	at org.apache.spark.sql.sedona_sql.expressions.ST_Distance$$anonfun$$lessinit$greater$1.apply(Functions.scala:39)
	at org.apache.spark.sql.sedona_sql.expressions.InferredBinaryExpression.evalWithoutSerialization(NullSafeExpressions.scala:285)
	at org.apache.spark.sql.sedona_sql.expressions.InferredBinaryExpression.eval(NullSafeExpressions.scala:279)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.co

Py4JJavaError: An error occurred while calling o72.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 6 in stage 48.0 failed 1 times, most recent failure: Lost task 6.0 in stage 48.0 (TID 195) (910c62cf2746 executor driver): java.lang.NoClassDefFoundError: org/opengis/referencing/NoSuchAuthorityCodeException
	at org.apache.spark.sql.sedona_sql.expressions.ST_Distance$$anonfun$$lessinit$greater$1.apply(Functions.scala:39)
	at org.apache.spark.sql.sedona_sql.expressions.ST_Distance$$anonfun$$lessinit$greater$1.apply(Functions.scala:39)
	at org.apache.spark.sql.sedona_sql.expressions.InferredBinaryExpression.evalWithoutSerialization(NullSafeExpressions.scala:285)
	at org.apache.spark.sql.sedona_sql.expressions.InferredBinaryExpression.eval(NullSafeExpressions.scala:279)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProjection.apply(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificUnsafeProj

## Visualization

In [ ]:
# Convert to pandas for visualization
points_pdf = filtered_points.toPandas()

# Create a simple scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(points_pdf['longitude'], points_pdf['latitude'], 
           c=points_pdf['value'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Value')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Points Distribution')
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Create an interactive map with Folium
if len(points_pdf) > 0:
    center_lat = points_pdf['latitude'].mean()
    center_lon = points_pdf['longitude'].mean()
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)
    
    for idx, row in points_pdf.iterrows():
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=5,
            popup=f"Name: {row['name']}<br>Value: {row['value']}",
            color='red',
            fill=True,
            fillColor='red'
        ).add_to(m)
    
    # Add bounding box
    folium.Rectangle(
        bounds=[[-10, -10], [10, 10]],
        color='blue',
        fill=False,
        popup='Bounding Box'
    ).add_to(m)
    
    m
else:
    print("No points found within the bounding box to display.")

## Cleanup

In [ ]:
# Stop Spark session
spark.stop()